Recherche et alignement : outil de recherche iconographique dans la base de données Gallica, Joconde et Europeana

Cet outil permet la récupération de données (textuelles et iconographiques) à partir de termes de recherche de la bases de données : Gallica, Joconde et Europeana

*Document d'entrée* : mot-clé.

*Documents de sortie* : Dans le dossier de travail, les fichiers de données ainsi que les fichiers téléchargés

In [16]:
#@markdown # Connexion du notebook à son compte Google Drive et signalement du dossier de travail :

'''
Google Colab notebook.
Python == 3.7.11

Marina Giardinetti
2024
'''

## Installation des bibliothèques et connexion au compte Google Drive

!pip install utils
from google.colab import drive
import json
from openpyxl import load_workbook
import pandas as pd
!pip install soup2dict
from soup2dict import convert
import ssl
import re
import requests
!pip install langdetect
from langdetect.lang_detect_exception import LangDetectException
import shutil
!pip install xmltojson
import xmltodict
import urllib
import urllib.request, urllib.error, urllib.parse
import xmltojson
from langdetect import detect
import os
import xml.etree.ElementTree as ET
import lxml
from collections import OrderedDict
from lxml import etree
!pip install unidecode==1.2.0
import unidecode
import folium
import unicodedata
from urllib.error import HTTPError
from re import match
from bs4 import BeautifulSoup
import urllib.request, urllib.error, urllib.parse
import string
import time


!pip uninstall pyOpenSSL==0.15.1 -y
!pip install pyOpenSSL==17.2.0

drive.mount('/content/drive/')


#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

#@markdown ####Indiquer le chemin vers le dossier de travail sur le Google Drive (si le dossier n'existe pas encore, il sera créé lors de l'exécution de la cellule) :
chemin_vers_dossier_travail = '/content/drive/My Drive/BNF_15_05' # @param {type:"string"}

if not os.path.exists(chemin_vers_dossier_travail):
    os.makedirs(chemin_vers_dossier_travail)
os.chdir(chemin_vers_dossier_travail)

from urllib.error import HTTPError

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
terme_a_chercher = "ordinateur" # @param {type:"string"}
os.chdir(chemin_vers_dossier_travail)
os.makedirs(terme_a_chercher, exist_ok=True)
os.chdir(terme_a_chercher)


chemin_dossier = "".join(chemin_vers_dossier_travail+'/'+terme_a_chercher)



Found existing installation: pyOpenSSL 17.2.0
Uninstalling pyOpenSSL-17.2.0:
  Successfully uninstalled pyOpenSSL-17.2.0
  Using cached pyOpenSSL-17.2.0-py2.py3-none-any.whl (52 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.6.3 requires pyOpenSSL>=19.1.0, but you have pyopenssl 17.2.0 which is incompatible.
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [17]:
#@markdown # Gallica

os.chdir(chemin_dossier)
dossier_base_gallica = 'images_gallica'
os.makedirs(dossier_base_gallica, exist_ok=True)
new_gallica = {}

if ' ' in terme_a_chercher:
  nom_pour_lien = re.sub(r' ', '%20', terme_a_chercher)
  lien_de_recherche_gallica = "".join(["https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.title%20adj%20", nom_pour_lien, "&maximumRecords=49&deway=%20any%207"])
else:
  lien_de_recherche_gallica = "".join(["https://gallica.bnf.fr/SRU?version=1.2&operation=searchRetrieve&query=dc.title%20adj%20"+terme_a_chercher+'&maximumRecords=49&deway=%20any%207'])

time.sleep(5)
response = requests.get(lien_de_recherche_gallica, verify=False, headers=headers, timeout=5)

parse = BeautifulSoup(response.content,'html.parser')
with open ('fichier_temporaire.xml', 'w') as pu:
 pu.write(parse.prettify())
with open('fichier_temporaire.xml', 'r') as po:
  my =xmltodict.parse(po.read())
  my2d = dict(OrderedDict(my))
a_suppr = "".join(chemin_vers_dossier_travail+'/'+terme_a_chercher+'/fichier_temporaire.xml')
os.remove(a_suppr)
#my2 = xmltodict.parse(my2d)
for ele in my2d['srw:searchretrieveresponse']['srw:records']['srw:record']:
  for key, value in ele.items():
    if key == 'srw:recorddata':
      for key2, value2 in value.items():
        for key3, value3 in value2.items():
          if key3 == 'dc:title':
            le_titre = str(value3)
            le_nouveau_titre = re.sub(r"[\['\]]", '', le_titre)
            new_gallica[le_nouveau_titre] = {}
          if key3 == 'dc:date':
            date = value3
          if key3 == 'dc:creator':
            auteur = value3
          if key3 == 'dc:type':
            types = value3
          if key3 == 'dc:subject':
            subject = value3
          if key3 == 'dc:identifier':
            if type(value3) == list:
              value3 = value3[0]
              identifiant = re.findall('b[a-z0-9]{4,}', value3)
            else:
              identifiant = re.findall('b[a-z0-9]{4,}', value3)
            for id in identifiant:
              lien = 'http://gallica.bnf.fr/iiif/ark:/12148/' + id + '/f1/full/3000/0/native.jpg'
              nom_image = "".join([chemin_vers_dossier_travail+'/'+terme_a_chercher+ '/', dossier_base_gallica, '/', id + '.jpg'])
              time.sleep(15)
              r=requests.get(lien, headers=headers, stream=True)
              with open(nom_image, 'wb') as f:
                f.write(r.content)
      try:
        new_gallica[le_nouveau_titre] = {'Date': date, 'Auteur':auteur, 'Type': types, 'Sujet': subject, 'Lien': lien}
      except NameError:
        try:
          auteur
          subject
          date
          types
        except NameError:
          auteur = ''
          subject = ''
          date = ''
          types = ''
          new_gallica[le_nouveau_titre] = {'Date': date, 'Auteur':auteur, 'Type': types, 'Sujet': subject, 'Lien': lien}


df_gallica = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in new_gallica.items()]))
df_gallica = df_gallica.transpose()

df_gallica.to_excel('donnees_gallica.xlsx')

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gallica.bnf.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [21]:
#@markdown # Europeana
from json.decoder import JSONDecodeError
os.chdir(chemin_dossier)
dossier_base_europeana = 'images_europeana'
os.makedirs(dossier_base_europeana, exist_ok=True)
new_europeana = {}

if ' ' in terme_a_chercher:
  nom_pour_lien = re.sub(r' ', '%20', terme_a_chercher)
  lien_de_recherche_europeana = "".join(["https://api.europeana.eu/record/v2/search.json?query="+nom_pour_lien+'&rows=100&theme=art&wskey=alemalle'])
else:
  lien_de_recherche_europeana = "".join(["https://api.europeana.eu/record/v2/search.json?query="+terme_a_chercher+'&rows=100&theme=art&wskey=alemalle'])
request = urllib.request.Request(lien_de_recherche_europeana)
try:
  content = urllib.request.urlopen(request)
except HTTPError as he:
  print('Pas de téléchargement possible pour la base Europeana avec cette recherche')
parse = BeautifulSoup(content,'html.parser')
json_infos=json.loads(parse.text)

ssl._create_default_https_context = ssl._create_unverified_context
for keys, values in json_infos.items():
  if keys == 'items':
    for ele in values:
      for keys2, values2 in ele.items():
        if keys2 == 'title':
          if type(values2) == list:
            titre_trouve = values2[0]
          nouveau_titre_trouve = re.sub(r"[\['\]]",'',  titre_trouve)
        if keys2 == 'dcCreator':
            auteurs = str(values2)
        if keys2 == 'edmConceptPrefLabelLangAware':
          for keys3, values3 in values2.items():
            if keys3 == 'fr':
              typee = str(values3)
        if keys2 == 'edmConceptLabel':
          for ele2 in values2:
            for keys4, values4 in ele2.items():
              try:
                lang = detect(values4)
              except LangDetectException as lde:
                pass
              if lang == 'fr':
                sujets = str(values4)
        if keys2 == 'edmIsShownBy':
          for ele2 in values2:
            lien = ele2
            nom = re.sub('/', '_', lien)
            try:
              titre_pour_nom = re.sub('\/', '_', nouveau_titre_trouve)
              nom_image2 = ''.join([chemin_vers_dossier_travail+'/'+terme_a_chercher+ '/', dossier_base_europeana, '/', titre_pour_nom + '.jpg'])
              if 'gallica' in lien:
                pass
              else:
                r=requests.get(lien, headers=headers, stream=True)
                with open(nom_image2, 'wb') as f:
                  f.write(r.content)
         #   urllib.request.urlretrieve(lien, nom_image2)
            except (NameError, requests.exceptions.SSLError) :pass
      try:
        new_europeana[nouveau_titre_trouve]= {'Auteur': auteurs, 'Type': typee, 'Sujet': sujets, 'Lien': lien}
      except NameError:
          auteurs = ''
          new_europeana[nouveau_titre_trouve]= {'Auteur': auteurs, 'Type': typee, 'Sujet': sujets, 'Lien': lien}

df_europeana = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in new_europeana.items()]))
df_europeana = df_europeana.transpose()

df_europeana.to_excel('donnees_europeana.xlsx')

JSONDecodeError: Expecting ',' delimiter: line 1 column 72498 (char 72497)

In [15]:
#@markdown # Base Joconde
os.chdir(chemin_dossier)
new_joconde = {}
auteurss, sujetss,materiaux_techniques, datess = ([] for i in range(4))

if ' ' in terme_a_chercher:
  nom_pour_lien = re.sub(r' ', '%20', terme_a_chercher)
  lien_de_recherche_joconde = "".join("https://data.culture.gouv.fr/api/records/1.0/search/?dataset=base-joconde-extrait&q="+nom_pour_lien+'&rows=10000')
else:
  lien_de_recherche_joconde="".join("https://data.culture.gouv.fr/api/records/1.0/search/?dataset=base-joconde-extrait&q="+terme_a_chercher+'&rows=10000')
request = urllib.request.Request(lien_de_recherche_joconde)
content = urllib.request.urlopen(request)
parse = BeautifulSoup(content,'html.parser')
donnees_joconde=json.loads(parse.text)

for key, value in donnees_joconde.items():
  if key == 'records':
    for ele in value:
      for key2, value2 in ele.items():
        if key2 == 'fields':
          for key3, value3 in value2.items():
            if key3 == 'titre':
              titre = str(value3)
            if key3 == 'auteur':
              auteurss = value3
            if key3 == 'materiaux_techniques':
              materiaux_techniques = value3
            if key3 == 'sujet_represente':
              sujetss = value3
            if key3 == 'epoque':
              datess = str(value3)
            if key3 == 'periode_de_creation':
              datess = value3
      try:
        new_joconde[titre]={'Date': datess, 'Auteur':auteurss, 'Matériaux': materiaux_techniques, 'Sujet': sujetss}
      except NameError:pass

df_joconde = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in new_joconde.items()]))
df_joconde = df_joconde.transpose()
df_joconde.to_excel('donnees_joconde.xlsx')

